In [1]:
# !pip install azure-cognitiveservices-personalizer

In [1]:
from azure.cognitiveservices.personalizer import PersonalizerClient
from azure.cognitiveservices.personalizer.models import RankableAction, RewardRequest, RankRequest
from msrest.authentication import CognitiveServicesCredentials
import datetime, json, os, time, uuid

In [4]:
# !export PERSONALIZER_RESOURCE_KEY="5627576870194e978b5f9c753c853a0f"
# !export PERSONALIZER_RESOURCE_ENDPOINT="https://personalizer-resource.cognitiveservices.azure.com/"
# !source ~/.bashrc

# key_var_name = 'PERSONALIZER_RESOURCE_KEY'
# if not key_var_name in os.environ:
#     raise Exception('Please set/export the environment variable: {}'.format(key_var_name))
# personalizer_key = os.environ[key_var_name]

# # Replace <your-resource-name>: https://<your-resource-name>.api.cognitive.microsoft.com/
# endpoint_var_name = 'PERSONALIZER_RESOURCE_ENDPOINT'
# if not endpoint_var_name in os.environ:
#     raise Exception('Please set/export the environment variable: {}'.format(endpoint_var_name))
# personalizer_endpoint = os.environ[endpoint_var_name]

In [8]:
key="5627576870194e978b5f9c753c853a0f"
endpoint="https://personalizer-resource.cognitiveservices.azure.com/"

In [9]:
client = PersonalizerClient(endpoint, CognitiveServicesCredentials(key))

In [10]:
def get_actions():
    action1 = RankableAction(id='pasta', features=[{"taste":"salty", "spice_level":"medium"},{"nutrition_level":5,"cuisine":"italian"}])
    action2 = RankableAction(id='ice cream', features=[{"taste":"sweet", "spice_level":"none"}, { "nutritional_level": 2 }])
    action3 = RankableAction(id='juice', features=[{"taste":"sweet", 'spice_level':'none'}, {'nutritional_level': 5}, {'drink':True}])
    action4 = RankableAction(id='salad', features=[{'taste':'salty', 'spice_level':'none'},{'nutritional_level': 2}])
    return [action1, action2, action3, action4]

In [11]:
def get_user_timeofday():
    res={}
    time_features = ["morning", "afternoon", "evening", "night"]
    time = input("What time of day is it (enter number)? 1. morning 2. afternoon 3. evening 4. night\n")
    try:
        ptime = int(time)
        if(ptime<=0 or ptime>len(time_features)):
            raise IndexError
        res['time_of_day'] = time_features[ptime-1]
    except (ValueError, IndexError):
        print("Entered value is invalid. Setting feature value to", time_features[0] + ".")
        res['time_of_day'] = time_features[0]
    return res

In [12]:
def get_user_preference():
    res = {}
    taste_features = ['salty','sweet']
    pref = input("What type of food would you prefer? Enter number 1.salty 2.sweet\n")
    
    try:
        ppref = int(pref)
        if(ppref<=0 or ppref>len(taste_features)):
            raise IndexError
        res['taste_preference'] = taste_features[ppref-1]
    except (ValueError, IndexError):
        print("Entered value is invalid. Setting feature value to", taste_features[0]+ ".")
        res['taste_preference'] = taste_features[0]
    return res

The Personalizer learning loop is a cycle of rank and reward calls. In this quickstart, each rank call, to personalize the content, is followed by a reward call to tell Personalizer how well the service ranked the content.

In [18]:
keep_going = True
while keep_going:

    eventid = str(uuid.uuid4())

    context = [get_user_preference(), get_user_timeofday()]
    actions = get_actions()

    rank_request = RankRequest( actions=actions, context_features=context, excluded_actions=['juice'], event_id=eventid)
    response = client.rank(rank_request=rank_request)
    
    print("Personalizer service ranked the actions with the probabilities listed below:")
    
    rankedList = response.ranking
    for ranked in rankedList:
        print(ranked.id, ':',ranked.probability)

    print("Personalizer thinks you would like to have", response.reward_action_id+".")
    answer = input("Is this correct?(y/n)\n")[0]

    reward_val = "0.0"
    if(answer.lower()=='y'):
        reward_val = "10.0"
    elif(answer.lower()=='n'):
        reward_val = "0.0"
    else:
        print("Entered choice is invalid. Service assumes that you didn't like the recommended food choice.")

    client.events.reward(event_id=eventid, value=reward_val)

    br = input("Press Q to exit, any other key to continue: ")
    if(br.lower()=='q'):
        keep_going = False

What type of food would you prefer? Enter number 1.salty 2.sweet
1
What time of day is it (enter number)? 1. morning 2. afternoon 3. evening 4. night
2
Personalizer service ranked the actions with the probabilities listed below:
ice cream : 0.333333343
pasta : 0.333333343
juice : 0.0
salad : 0.333333343
Personalizer thinks you would like to have ice cream.
Is this correct?(y/n)
y
Press Q to exit, any other key to continue: 1
What type of food would you prefer? Enter number 1.salty 2.sweet
1
What time of day is it (enter number)? 1. morning 2. afternoon 3. evening 4. night
2
Personalizer service ranked the actions with the probabilities listed below:
salad : 0.333333343
ice cream : 0.333333343
juice : 0.0
pasta : 0.333333343
Personalizer thinks you would like to have salad.
Is this correct?(y/n)
n
Press Q to exit, any other key to continue: 1
What type of food would you prefer? Enter number 1.salty 2.sweet
1
What time of day is it (enter number)? 1. morning 2. afternoon 3. evening 4. ni

In [19]:
response = client.rank(rank_request=rank_request)
rankedList = response.ranking
for ranked in rankedList:
    print(ranked.id, ':',ranked.probability)

salad : 0.8666667
ice cream : 0.06666667
juice : 0.0
pasta : 0.06666667
